# USE HMM network to inference 

1. Since max step =2, each room can generate a HMM with neibours room in previous time state and relevant sensors at current and all previous  time stamp.
2. we use a binary situation for outcome space {0,1} 
3. Based on the network, we will learn P(X_1={0/1} | X ={0/1}}, note the state will determine the action directly
4. Then we can inference B(X_t) by passage of time and observations Topic 8 from page 28


In [124]:
from __future__ import division
from __future__ import print_function

# Allowed libraries 
import numpy as np
import pandas as pd
import scipy as sp
import scipy.special
import heapq as pq
import matplotlib as mp
import matplotlib.pyplot as plt
import math
from itertools import product, combinations
from collections import OrderedDict as odict
import collections
from graphviz import Digraph, Graph
from tabulate import tabulate
import copy
import sys
import os
import datetime
import sklearn
import ast
import re

In [125]:
data = pd.read_csv("data.csv")

In [126]:
data.columns

Index(['Unnamed: 0', 'reliable_sensor1', 'reliable_sensor2',
       'reliable_sensor3', 'reliable_sensor4', 'unreliable_sensor1',
       'unreliable_sensor2', 'unreliable_sensor3', 'unreliable_sensor4',
       'robot1', 'robot2', 'door_sensor1', 'door_sensor2', 'door_sensor3',
       'door_sensor4', 'time', 'electricity_price', 'r1', 'r2', 'r3', 'r4',
       'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15',
       'r16', 'r17', 'r18', 'r19', 'r20', 'r21', 'r22', 'r23', 'r24', 'r25',
       'r26', 'r27', 'r28', 'r29', 'r30', 'r31', 'r32', 'r33', 'r34', 'r35',
       'c1', 'c2', 'c3', 'c4', 'o1', 'outside'],
      dtype='object')

In [127]:
data_copy = copy.deepcopy(data)

In [128]:
MAP = {
    "r1":["r2","r3"],
    "r2":["r1","r4"],
    "r3":["r1","r7"],
    "r4":["r2","r8"],
    "r5":["r6","r9","c3"],
    "r6":["r5","c3"],
    "r7":["r3","c1"],
    "r8":["r4","r9"],
    "r9":["r5","r8","r13"],
    "r10":["c3"],
    "r11":["c3"],
    "r12":["outside","r22"],
    "r13":["r9","r24"],
    "r14":["r24"],
    "r15":["c3"],
    "r16":["c3"],
    "r17":["c3"],
    "r18":["c3"],
    "r19":["c3"],
    "r20":["c3"],
    "r21":["c3"],
    "r22":["r12","r25"],
    "r23":["r24"],
    "r24":["r13","r14","r23"],
    "r25":["r22","r26","c1"],
    "r26":["r25","r27"],
    "r27":["r26","r32"],
    "r28":["c4"],
    "r29":["c4","r30"],
    "r30":["r29"],
    "r31":["r32"],
    "r32":["r27","r31","r33"],
    "r33":["r32"],
    "r34":["c2"],
    "r35":["c4"],
    "c1":["r7","r25","c2"],
    "c2":["c1","r34","c4"],
    "c3": ["r5","r6","r10","r11","r15","r16","r17","r18","r19","r20","r21","o1"],
    "c4":["r29","c2","r35","r28","o1"],
    "o1":["c3","c4"],
    "outside":["r12"]  
}
censor_dict =  {
    "r1": "us3", #us for unreliable_sensor
    "r5": "rs2" ,#rs for reliable_sensor
    "r8": "ds1", #ds for door sensor
    "r9": "ds1",
    "r16": "rs1",
    "r24" : "us4",
    "r25" : "rs3",
    "r26" : "ds3", 
    "r27" : "ds3",
    "r31" : "rs4",
    "r35" : "ds4",
    "c1" : "ds2",
    "c2" : "ds2",
    "c3" : "us2",
    "c4" : "ds4",
    "o1" : "us1"
}

In [129]:
def n_step_neighbour(node,n,G):
    neighbour_list = []
    neighbour_list = neighbour_list + G[node]
    while n > 1:
        for new_node in neighbour_list:
            neighbour_list = neighbour_list+ G[new_node]
        n = n -1 
    n_list = list(set(neighbour_list))
    n_list.remove(node)
    return(n_list)

In [131]:
def generate_HMM(node,G,censors):
    HMM = {}
    previous_node = "old_" + node
    node_neighbour = n_step_neighbour(node,2,G)
    HMM = {i:[node] for i in node_neighbour}
    HMM[previous_node] = [node]
    
    related_censor = [censors.get(node)] + [censors.get(i) for i in node_neighbour]
    related_censor = [x for x in related_censor  if x is not None]
    
    for i in related_censor:
        if i[0] =="d":
            if sum([censor == i for censor in related_censor]) < 2:
                related_censor.remove(i) #drop door censor when you only at one side
    
    HMM[node] = list(set(related_censor))
    
    
    return(HMM)
    
    
#def learn_prob(G,data):
    

In [160]:
def learn_tranisition(curr_node,pre_node,data):
    new_df = pd.DataFrame()
    new_df['curr'] = data[curr_node][1:]
    new_df['pre'] = data[pre_node].shift(1)[:-1]
    
    
    prob_ct = pd.crosstab(new_df['pre'] > 0 , new_df['curr'] > 0, normalize = "index")
    
    tran_table = odict()
    
    tran_table[(1,1)] = prob_ct.loc[True,True] # p(current = 1 | previous = 1)
    tran_table[(1,0)] =prob_ct.loc[False,True]# p(current = 1 | previous = 0)
    tran_table[(0,1)] = prob_ct.loc[True,False]
    tran_table[(0,0)] =  prob_ct.loc[False,False]
    
    
    return ({'dom': (curr_node,pre_node), 'table':tran_table})

def learn_censor_prob(node, censor, data):
    
    prob_ct = pd.crosstab(data[node] > 0 , data[censor] == "motion", normalize = "index")
    
    tran_table = odict()
    
    tran_table[(1,1)] = prob_ct.loc[True,True] # p(current = 1 | previous = 1)
    tran_table[(1,0)] =prob_ct.loc[False,True]# p(current = 1 | previous = 0)
    tran_table[(0,1)] = prob_ct.loc[True,False]
    tran_table[(0,0)] =  prob_ct.loc[False,False]
    
    return ({'dom': (censor,node), 'table':tran_table})

In [159]:
learn_censor_prob('r25','reliable_sensor3',data)

{'dom': ('r25', 'reliable_sensor3'),
 'table': OrderedDict([((1, 1), 0.9568261376896149),
              ((1, 0), 0.03367875647668394),
              ((0, 1), 0.043173862310385065),
              ((0, 0), 0.966321243523316)])}

For one room:
    1. generate the network 
    2. Assign the conditional prob (factor tables) 
    3. Find P(X=1| other nodes, e) and P(X=0| other nodes, e) by HMM algorithm;
    4. Make decicions (threshold) 
 
Entire inference process: 
    1. calculate all the probabilities 
    2. For t = 1, ... T: 
        a. Do inference for each room
        b. Store the decision 
        c. move to next timestep 
             
  
  